In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from src.robot_reboot.dataset_factory import RobotRebootDataSetFactory

In [2]:
cnn = tf.keras.models.load_model('model/model_0')
cnn.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 31, 31, 9)]  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 31, 31, 64)   9280        input[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 15, 15, 64)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 15, 15, 64)   65600       max_pooling2d_6[0][0]            
____________________________________________________________________________________________

In [3]:
ds_factory = RobotRebootDataSetFactory(31, cnn, max_depth=20, seed=26, playouts=50)

In [4]:
rows, cols, layers = 31, 31, 9
robots = int((layers - 1)/2)
p_columns = [f'p_{i}' for i in range(robots * 4)]
maze_columns = [f'm_{i}' for i in range(rows * cols *layers)]
columns = ['v'] + p_columns + maze_columns

In [5]:
def write_tf_record(ds_factory, max_movements, i, f):
    v, p, s = ds_factory.create(locate_robot_close_goal=True, max_movements=max_movements)
    state = s.reshape(rows * cols * layers,)
    sample = tf.train.Example(
        features = tf.train.Features(
            feature = {
                    'v': tf.train.Feature(float_list=tf.train.FloatList(value=[v])),
                    'p': tf.train.Feature(float_list=tf.train.FloatList(value = p)),
                    's': tf.train.Feature(float_list=tf.train.FloatList(value = state))
                }
            )
    )
    f.write(sample.SerializeToString())

In [ ]:
from threading import Thread

np.random.seed(26)
def create_tfrecord(batch, n):
    threads = list()
    with tf.io.TFRecordWriter(f'robot_reboot_{batch}.tfrecord') as f:
        for i in range(0, n):
            if i% 10 ==0 and i!=0:
                
                [t.join() for t in threads]
                threads = list()
            max_movements = np.random.randint(1, 6)
            thread = Thread(target = write_tf_record, args=(ds_factory, max_movements, i, f))
            thread.start()
            threads.append(thread)
        [t.join() for t in threads]

for i in range(0, 30):
    print(f'Creating batch {i + 1}/30')
    create_tfrecord(i + 1, 20)

Creating batch 1/30
Creating batch 2/30
Creating batch 3/30
Creating batch 4/30
Creating batch 5/30
Creating batch 6/30
Creating batch 7/30


Exception in thread Thread-137:
Traceback (most recent call last):
  File "/usr/local/opt/python/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/local/opt/python/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-6f7ab89a8e79>", line 2, in write_tf_record
    v, p, s = ds_factory.create(locate_robot_close_goal=True, max_movements=max_movements)
  File "/Users/danielavelasquez/mlearning/robot-reboot/src/robot_reboot/dataset_factory.py", line 59, in create
    p = mcts.search(state)
  File "/Users/danielavelasquez/mlearning/robot-reboot/src/alphazero/montecarlo_tree_search.py", line 73, in search
    p[i] = self.__simulations(next_state, self.__playouts)
  File "/Users/danielavelasquez/mlearning/robot-reboot/src/alphazero/montecarlo_tree_search.py", line 84, in __simulations
    v = np.array([self.__play

Creating batch 8/30
Creating batch 9/30
Creating batch 10/30
Creating batch 11/30
Creating batch 12/30
